# Задача регрессии

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import linear_model

## Первичный анализ данных

In [3]:
df_vacancy = pd.read_csv('hh_vacancy.csv')
df_vacancy

,id,area_id,area_name,employer_id,employment_id,employment_name,experience_id,experience_name,key_skills,name,published_at,salary_currency,salary_from,salary_to,schedule_id,schedule_name,specialization_id,specialization_name
0,25607531,1,Москва,1747017.0,full,Полная занятость,between3And6,От 3 до 6 лет,"PHP, Symfony, PostgreSQL, Redis, Design Patter...",Senior/Middle PHP-Developer,2018-05-06T17:43:45+0300,RUR,120000.0,170000.0,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
1,25435136,1,Москва,553714.0,project,Проектная работа,between3And6,От 3 до 6 лет,NaN,Ведущий фронтенд-разработчик,2018-05-05T14:43:01+0300,NaN,NaN,NaN,remote,Удаленная работа,1,"Информационные технологии, интернет, телеком"
2,25284500,84,Ставрополь,5402.0,full,Полная занятость,between3And6,От 3 до 6 лет,"JavaScript, HTML5, CSS3, AngularJS, Bootstrap",Frontend-разработчик,2018-05-04T09:24:18+0300,RUR,60000.0,80000.0,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
3,25641559,1,Москва,1554.0,full,Полная занятость,between1And3,От 1 года до 3 лет,"1С программирование, 1С: Бухгалтерия, 1С Управ...",Программист-консультант 1С,2018-05-06T10:49:05+0300,NaN,NaN,NaN,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
4,25527035,2,Санкт-Петербург,3289100.0,full,Полная занятость,between1And3,От 1 года до 3 лет,NaN,Менеджер по продажам,2018-05-05T16:15:16+0300,RUR,35000.0,100000.0,flexible,Гибкий график,1,"Информационные технологии, интернет, телеком"
5,25178591,1,Москва,2353123.0,full,Полная занятость,between3And6,От 3 до 6 лет,NaN,Системный аналитик,2018-05-07T00:54:39+0300,NaN,NaN,NaN,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
6,25449125,1,Москва,934019.0,full,Полная занятость,between3And6,От 3 до 6 лет,NaN,Java-разработчик,2018-05-06T11:41:28+0300,RUR,120000.0,220000.0,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
7,25459894,1,Москва,1053428.0,full,Полная занятость,between3And6,От 3 до 6 лет,"Linux, AWS, Proxmox, Vagrant, Docker, Админист...",Системный администратор (DevOps),2018-05-06T17:00:04+0300,RUR,100000.0,NaN,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
8,25041175,2,Санкт-Петербург,934019.0,full,Полная занятость,between1And3,От 1 года до 3 лет,NaN,Java разработчик (middle/senior),2018-05-05T18:43:16+0300,RUR,120000.0,220000.0,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
9,25295125,78,Самара,2618807.0,full,Полная занятость,between1And3,От 1 года до 3 лет,"Коммуникабельность, Аналитический склад ума, Н...",Менеджер по работе с корпоративными клиентами,2018-05-06T11:45:27+0300,RUR,35000.0,NaN,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"


In [4]:
df_vacancy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19837 entries, 0 to 19836
Data columns (total 18 columns):
id                     19837 non-null int64
area_id                19837 non-null int64
area_name              19837 non-null object
employer_id            19813 non-null float64
employment_id          19805 non-null object
employment_name        19805 non-null object
experience_id          19805 non-null object
experience_name        19805 non-null object
key_skills             6654 non-null object
name                   19837 non-null object
published_at           19837 non-null object
salary_currency        10358 non-null object
salary_from            9611 non-null float64
salary_to              4594 non-null float64
schedule_id            19805 non-null object
schedule_name          19805 non-null object
specialization_id      19837 non-null int64
specialization_name    19837 non-null object
dtypes: float64(3), int64(3), object(12)
memory usage: 2.7+ MB


In [5]:
# Разделение на обучающую и тестовую выборку по id, который соответствует порядку добавления вакаансий на сайт.
count_df = len(df_vacancy)
df_vacancy.sort_values(by='id', ascending=True, inplace=True)
df_vacancy_test = df_vacancy[int(2*count_df/3):]
df_vacancy_train = df_vacancy[:int(2*count_df/3)]

In [6]:
# Первичный анализ данных и формирование признаков рекомендуется проводить на обучающей выборке.
df_vacancy_train.describe()

,id,area_id,employer_id,salary_from,salary_to,specialization_id
count,1.322400e+04,13224.000000,1.321900e+04,6555.000000,2695.000000,13224.000000
mean,2.427900e+07,339.665910,6.285927e+05,52182.384592,85212.069388,3.595886
std,1.580547e+06,800.582578,8.361471e+05,35127.270832,59299.426350,4.907250
min,7.160726e+06,1.000000,1.900000e+01,25.000000,30.000000,1.000000
25%,2.405097e+07,1.000000,4.114400e+04,30000.000000,40000.000000,1.000000
50%,2.489626e+07,17.000000,8.458500e+04,43000.000000,70000.000000,1.000000
75%,2.524230e+07,88.000000,9.994420e+05,64000.000000,120000.000000,4.000000
max,2.540689e+07,5962.000000,3.353801e+06,300000.000000,1000000.000000,29.000000


In [7]:
df_company = pd.read_csv('hh_company.csv')
df_company.head()

,employer_id,employer_name,employer_site,industry_id,industry_name
0,1747017.0,Колтач Солюшнс,NaN,7,"Информационные технологии, системная интеграци..."
1,553714.0,Нотамедиа,NaN,7,"Информационные технологии, системная интеграци..."
2,5402.0,НЕОЛАНТ,NaN,7,"Информационные технологии, системная интеграци..."
3,1554.0,Infosuite,NaN,7,"Информационные технологии, системная интеграци..."
4,3289100.0,Медиа Бокс,NaN,7,"Информационные технологии, системная интеграци..."


## Подготовка данных

In [8]:
# Удаление записей с пропущенными значениями NaN.
df_vacancy_train = df_vacancy_train[np.isfinite(df_vacancy_train['salary_from']) &
                                    np.isfinite(df_vacancy_train['area_id']) &
                                    np.isfinite(df_vacancy_train['employer_id']) &
                                    np.isfinite(df_vacancy_train['specialization_id'])]

In [9]:
train_y = df_vacancy_train['salary_from']
train_X = df_vacancy_train[['area_id', 'employer_id', 'specialization_id']]

## Обучение модели

In [10]:
reg = linear_model.LinearRegression()
reg.fit(train_X, train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [11]:
# Коэф. уравнения.
reg.coef_

array([-2.44679546e+00, -3.34828989e-03, -1.54451998e+03])

## Оценка качества обучения

In [12]:
# Точность на обучающей выборке.
train_r2 = r2_score(train_y, reg.predict(train_X))
train_r2

0.06585492603413479

In [13]:
# Подготовка тестовой выборки.
df_vacancy_test = df_vacancy_test[np.isfinite(df_vacancy_test['salary_from']) &
                                  np.isfinite(df_vacancy_test['area_id']) &
                                  np.isfinite(df_vacancy_test['employer_id']) &
                                  np.isfinite(df_vacancy_test['specialization_id'])]
test_y = df_vacancy_test['salary_from']
test_X = df_vacancy_test[['area_id', 'employer_id', 'specialization_id']]
# Точность на тестовой выборке.
test_r2 = r2_score(test_y, reg.predict(test_X))
test_r2

0.021286993703545587

In [17]:
def adjusted_r2_score(test_y, test_X, predict_y):
    '''Коэффициент детерминации, скорректированный на число признаков.'''
    n=test_X.shape[0]       # количество наблюдений
    p=test_X.shape[1] - 1   # количество признаков, включенных в модель
    r2 = r2_score(test_y, predict_y)
    adj_r2 = 1 - (1 - r2) * ((n - 1)/(n-p-1))
    return adj_r2

# Скорректированная точность на тестовой выборке.
test_adj_r2 = adjusted_r2_score(test_y, test_X, reg.predict(test_X))
test_adj_r2

0.02064289169216249

Качество модели очень плохое, поскольку признаков мало и использован простейший способ кодирования категорильнаых признаков, который не подходит для линейны модели. Необходимиый результат коэф. детерминации 0.7